In [19]:
import requests
import json
import pandas as pd
import os
import cx_Oracle

In [95]:
#connection = cx_Oracle.connect('HCIN_GUPTAN1[AP_UWI]/aGqQ4jnX92NG@INCL02.IN.PROD/HWIN_USR_DEV.HOMECREDIT.IN')
#cursor = connection.cursor()
#os.chdir(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\Common Maps KNN")
#qu1= """
#select * from tmp_bd_grid_distance_id_1
#where time_decision_date between '01-JAN-19' and '01-MAR-19'
#"""
#df = pd.read_sql(qu1,connection)
#df.to_pickle(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\Atlan POC\data_dump.pkl")
#df.to_csv(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\Atlan POC\data_dump.csv")
df = pd.read_csv(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\Atlan POC\data_dump_bangalore.csv")


In [96]:
#select a.* from tmp_bd_grid_distance_id_1 a 
#join 
#(select * from uw_daily_data_all
#where upper(POS_CITY) in ('BANGALORE','BENGALURU')
#)b
#on a.skp_credit_case = b.skp_credit_case

In [102]:
df.RISK_FSTQPD30.value_counts()

0    39985
1     5437
Name: RISK_FSTQPD30, dtype: int64

In [103]:
#Downsampling Code 
seed = 1
#count_class_0, count_class_1 = df.RISK_FSTQPD30.value_counts()
count_class_0,count_class_1 = 5000, 5000
df_class_0 = df[df['RISK_FSTQPD30'] == 0]
df_class_1 = df[df['RISK_FSTQPD30'] == 1]
df_class_0_under = df_class_0.sample(count_class_0)
df_class_1_under = df_class_1.sample(count_class_1)
df_under = pd.concat([df_class_0_under, df_class_1_under], axis=0)

print('Random under-sampling:')
print(df_under.RISK_FSTQPD30.value_counts())
df_under.RISK_FSTQPD30.value_counts();

Random under-sampling:
1    5000
0    5000
Name: RISK_FSTQPD30, dtype: int64


In [113]:
df_under.shape

(10000, 10)

In [115]:
def api_call(skp_credit_case,lat,long):
    latlong = str(lat)+','+str(long)
    headers = {'X-API-KEY': '1DUQBXO5VCZQS3XS'}
    params = (('latlong', latlong),)
    response = requests.get('https://api-grid.atlan.com/api/profile/v1/india/super_bundle', headers=headers, params=params)
    data = response.text
    parsed = json.loads(data)
    df_tmp = pd.DataFrame(parsed['data'])
    df_tmp['SKP_CREDIT_CASE'] = skp_credit_case
    return df_tmp

In [116]:
x = api_call(0,12.9717,77.5945)
for i in range(0, df_under.shape[0]-1):
        y = api_call(df_under.iloc[i,0],df_under.iloc[i,2],df_under.iloc[i,3])
        x = x.append(y)

KeyError: 'data'

In [118]:
x.shape


(91, 84)

In [119]:
x.to_csv(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\Atlan POC\api_result.csv")

In [121]:
latlong = str(12.9717)+','+str(77.5945)
headers = {'X-API-KEY': '1DUQBXO5VCZQS3XS'}
params = (('latlong', latlong),)
response = requests.get('https://api-grid.atlan.com/api/profile/v1/india/super_bundle', headers=headers, params=params)
data = response.text
parsed = json.loads(data)
df_tmp = pd.DataFrame(parsed['data'])
df_tmp['SKP_CREDIT_CASE'] = skp_credit_case

KeyError: 'data'

In [123]:
data

'{\n    "message": "Limit exceeded"\n}\n'